In [29]:
# car pridiction model

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import pickle

In [62]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JBrGeRfYJT3wP7x3rkoyQ-oawrnMTJkSdqVeVfy2xpZb',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'carpricepridect-donotdelete-pr-fwtlvvfduaseap'
object_key = 'cardekho.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
print(df.shape)
df.sample(10)

(8128, 12)


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
3711,Tata New Safari DICOR 2.2 VX 4x2 BS IV,2012,300000,80000,Diesel,Individual,Manual,Second Owner,13.93,2179.0,138.1,7.0
2828,Audi Q3 35 TDI Quattro Premium Plus,2018,2800000,26000,Diesel,Dealer,Automatic,First Owner,15.73,1968.0,174.33,5.0
6331,Nissan Micra Diesel XV,2011,210000,110000,Diesel,Individual,Manual,Third Owner,23.08,1461.0,63.1,5.0
284,Maruti Alto LX,2005,110000,45000,Petrol,Individual,Manual,Second Owner,19.70,796.0,46.3,5.0
6711,Tata Manza Aura (ABS) Safire,2010,140000,116700,Petrol,Individual,Manual,Fourth & Above Owner,15.00,1368.0,90,5.0
5968,Maruti Ertiga SHVS VDI,2016,790000,120000,Diesel,Individual,Manual,First Owner,24.52,1248.0,88.5,7.0
1457,Mahindra Quanto C4,2016,403000,30000,Diesel,Individual,Manual,Second Owner,17.21,1493.0,100,7.0
5943,Maruti S-Cross 2017-2020 Delta DDiS 200 SH,2016,690000,32000,Diesel,Dealer,Manual,First Owner,25.10,1248.0,88.5,5.0
1736,Hyundai i20 Active S Petrol,2019,800000,5000,Petrol,Individual,Manual,First Owner,17.19,1197.0,81.86,5.0
3683,Hyundai i20 Sportz 1.4 CRDi,2014,420000,50000,Diesel,Individual,Manual,First Owner,22.54,1396.0,88.73,5.0


In [63]:
f_df=df.drop(['seller_type','engine','max_power','seats'],axis='columns')
f_df['mileage(km/ltr/kg)']=f_df['mileage(km/ltr/kg)'].fillna(f_df['mileage(km/ltr/kg)'].median())
f_df.head()

,name,year,selling_price,km_driven,fuel,transmission,owner,mileage(km/ltr/kg)
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Manual,First Owner,23.40
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Manual,Second Owner,21.14
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Manual,Third Owner,17.70
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Manual,First Owner,23.00
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Manual,First Owner,16.10


In [64]:
y=f_df.selling_price

In [65]:
x1=pd.get_dummies(f_df.name)
car_value_name=x1.drop_duplicates().reset_index(drop=True)
car_value_name.head()

,Ambassador CLASSIC 1500 DSL AC,Ambassador Classic 2000 DSZ AC PS,Ambassador Grand 1500 DSZ BSIII,Ambassador Grand 2000 DSZ PW CL,Ashok Leyland Stile LE,Audi A3 35 TDI Premium Plus,Audi A3 40 TFSI Premium,Audi A4 1.8 TFSI,Audi A4 2.0 TDI,Audi A4 2.0 TDI 177 Bhp Premium Plus,...,Volkswagen Vento Petrol Highline AT,Volkswagen Vento Petrol Trendline,Volvo S60 D4 SUMMUM,Volvo S90 D4 Inscription BSIV,Volvo V40 Cross Country D3,Volvo V40 D3 R-Design,Volvo XC40 D4 Inscription BSIV,Volvo XC40 D4 R-Design,Volvo XC60 Inscription D5 BSIV,Volvo XC90 T8 Excellence BSIV
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
#acess row value
id=np.array(car_value_name[car_value_name['BMW X4 M Sport X xDrive20d']==1]).flatten()
id1=np.array(car_value_name[car_value_name['Maruti Swift Dzire VDI']==1]).flatten()
id    

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [67]:
# mapping = {'First Owner': 4, 'Second Owner': 3, 'Third Owner': 2,
#            'Fourth & Above Owner': 1, 'Test Drive Car': 5}

mapping = {'First Owner': 4, 'Second Owner': 3, 'Third Owner': 2,
           'Fourth & Above Owner': 1, 'Test Drive Car': 5}
f_df.owner.unique()
f_df.owner=f_df.owner.map(mapping)

In [68]:
# year km_driven mileage(km/ltr/kg)
x2=f_df[['year', 'km_driven', 'mileage(km/ltr/kg)','owner']]
x2.head()

,year,km_driven,mileage(km/ltr/kg),owner
0,2014,145500,23.40,4
1,2014,120000,21.14,3
2,2006,140000,17.70,2
3,2010,127000,23.00,4
4,2007,120000,16.10,4


In [69]:
#fuel
x3=pd.get_dummies(f_df.fuel)
fuel=x3.drop_duplicates().reset_index(drop=True)
x3.head()

,CNG,Diesel,LPG,Petrol
0,0,1,0,0
1,0,1,0,0
2,0,0,0,1
3,0,1,0,0
4,0,0,0,1


In [70]:
#transmission
x4=pd.get_dummies(f_df.transmission)
transmission=x4.drop_duplicates().reset_index(drop=True)
x4.head()

,Automatic,Manual
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [71]:
#x1=Car name, x2=Nunaric columns(4), x3=fuel, x4=transmission and y= Price
x=pd.concat([x1,x2,x3,x4], axis=1)
x.head()

,Ambassador CLASSIC 1500 DSL AC,Ambassador Classic 2000 DSZ AC PS,Ambassador Grand 1500 DSZ BSIII,Ambassador Grand 2000 DSZ PW CL,Ashok Leyland Stile LE,Audi A3 35 TDI Premium Plus,Audi A3 40 TFSI Premium,Audi A4 1.8 TFSI,Audi A4 2.0 TDI,Audi A4 2.0 TDI 177 Bhp Premium Plus,...,year,km_driven,mileage(km/ltr/kg),owner,CNG,Diesel,LPG,Petrol,Automatic,Manual
0,0,0,0,0,0,0,0,0,0,0,...,2014,145500,23.40,4,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,2014,120000,21.14,3,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,2006,140000,17.70,2,0,0,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,2010,127000,23.00,4,0,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,2007,120000,16.10,4,0,0,0,1,0,1


In [72]:
model=linear_model.LinearRegression()
model.fit(x,y)
# x.isna().sum()
# f_df.isna().sum()

LinearRegression()

In [73]:

a=np.concatenate((id,np.array([2019,7500,16,4,0,1,0,0,1,0])))
a
model.predict([a])

/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([5457501.56599421])

In [74]:
model.score(x,y)

0.9884697062360259

In [75]:
with open('carPraicePridect_pickle','wb') as f:
    pickle.dump(model,f)

In [76]:
with open('carPraicePridect_pickle','rb') as f:
    mp=pickle.load(f)

In [77]:
mp.coef_

array([ -60590.37829257,  158522.53844424, -277643.87393652, ...,
        -68414.51705386,  407059.10180567, -407059.10095217])

In [78]:
mp.intercept_

-64040637.63949006